In [1]:
#Dependencies
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
case_files = glob.glob("Resources/Raw/*.csv")
case_files

['Resources/Raw/Diag_National.csv',
 'Resources/Raw/Diag_County.csv',
 'Resources/Raw/Diag_State.csv']

In [3]:
#Can't use a loop to read in csv's because they are uneven (specifically the county csv)
national_df = pd.read_csv('Resources/Raw/Diag_National.csv')
state_df = pd.read_csv('Resources/Raw/Diag_State.csv')
county_df = pd.read_csv('Resources/Raw/Diag_County.csv')
county_df.head()

,Indicator,Year,State,County,Geography,FIPS,Age Group,Race/Ethnicity,Sex,Transmission Category,Cases,Rate per 100000,Population
0,HIV diagnoses,2018,SC,Abbeville County,"Abbeville County, SC",45001,Ages 13 years and older,All races/ethnicities,Both sexes,All transmission categories,Data suppressed,Data suppressed,NaN
1,HIV diagnoses,2017,SC,Abbeville County,"Abbeville County, SC",45001,Ages 13 years and older,All races/ethnicities,Both sexes,All transmission categories,Data suppressed,Data suppressed,NaN
2,HIV diagnoses,2016,SC,Abbeville County,"Abbeville County, SC",45001,Ages 13 years and older,All races/ethnicities,Both sexes,All transmission categories,Data suppressed,Data suppressed,NaN
3,HIV diagnoses,2015,SC,Abbeville County,"Abbeville County, SC",45001,Ages 13 years and older,All races/ethnicities,Both sexes,All transmission categories,Data suppressed,Data suppressed,NaN
4,HIV diagnoses,2014,SC,Abbeville County,"Abbeville County, SC",45001,Ages 13 years and older,All races/ethnicities,Both sexes,All transmission categories,5,23.7,"21,075"


In [4]:
nat_df = national_df.iloc[:,[1,2,10]]
st_df = state_df.iloc[:,[1,2,10]]
cty_df = county_df.iloc[:,[1,4,12]]
# nat_df.head()
# st_df.head()
cty_df.head()

,Year,Geography,Population
0,2018,"Abbeville County, SC",NaN
1,2017,"Abbeville County, SC",NaN
2,2016,"Abbeville County, SC",NaN
3,2015,"Abbeville County, SC",NaN
4,2014,"Abbeville County, SC","21,075"


In [23]:
national_piv = nat_df.pivot(index='Geography',columns='Year', values='Population')
national_piv['Type'] = 'National'
national_piv

Year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Type
Geography,,,,,,,,,,,,
United States,"251,009,998","253,253,138","256,390,428","258,666,188","261,032,045","263,356,005","265,732,678","268,005,522","270,301,427","272,460,904","274,605,948",National


In [19]:
state_piv = st_df.pivot(index='Geography',columns='Year', values='Population')
state_piv['Type'] = 'State'
state_piv.head()

Year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Type
Geography,,,,,,,,,,,,
Alabama,"3,870,996","3,898,688","3,980,625","3,997,493","4,019,335","4,041,871","4,059,211","4,070,748","4,082,972","4,094,777","4,109,359",State
Alaska,"559,346","565,800","577,749","584,567","592,771","599,385","599,474","600,941","604,157","603,161","602,095",State
Arizona,"5,235,123","5,316,387","5,229,908","5,304,658","5,388,365","5,469,060","5,567,140","5,663,944","5,770,861","5,874,596","5,996,296",State
Arkansas,"2,357,363","2,374,089","2,407,988","2,426,548","2,438,682","2,449,550","2,459,895","2,470,760","2,482,955","2,495,932","2,508,847",State
California,"29,836,645","30,167,113","30,753,805","31,068,145","31,393,072","31,722,579","32,069,627","32,390,164","32,663,036","32,888,662","33,089,613",State


In [21]:
county_piv = cty_df.pivot(index='Geography',columns='Year', values='Population')
county_piv['Type'] = 'County'
county_piv.head()

Year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Type
Geography,,,,,,,,,,,,
"Abbeville County, SC",NaN,NaN,NaN,NaN,NaN,NaN,"21,075",NaN,NaN,NaN,NaN,County
"Acadia Parish, LA","48,028","48,081","49,736","49,793",NaN,"50,280","50,651","50,733","50,906","50,783","50,606",County
"Accomack County, VA",NaN,NaN,"28,124",NaN,NaN,"27,945","27,964","27,904","27,910",NaN,NaN,County
"Ada County, ID","306,746","311,241","317,604","325,159","332,649","340,146","349,356","356,854","368,126","379,392","391,848",County
"Adair County, IA","6,349","6,286","6,470","6,383","6,342","6,262","6,235","6,063","5,935","5,984","5,993",County


In [27]:
total_concat = pd.concat([national_piv, state_piv, county_piv])
total_concat.head()

Year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Type
Geography,,,,,,,,,,,,
United States,"251,009,998","253,253,138","256,390,428","258,666,188","261,032,045","263,356,005","265,732,678","268,005,522","270,301,427","272,460,904","274,605,948",National
Alabama,"3,870,996","3,898,688","3,980,625","3,997,493","4,019,335","4,041,871","4,059,211","4,070,748","4,082,972","4,094,777","4,109,359",State
Alaska,"559,346","565,800","577,749","584,567","592,771","599,385","599,474","600,941","604,157","603,161","602,095",State
Arizona,"5,235,123","5,316,387","5,229,908","5,304,658","5,388,365","5,469,060","5,567,140","5,663,944","5,770,861","5,874,596","5,996,296",State
Arkansas,"2,357,363","2,374,089","2,407,988","2,426,548","2,438,682","2,449,550","2,459,895","2,470,760","2,482,955","2,495,932","2,508,847",State


In [47]:
total_df = total_concat.reset_index()
total_df = total_df.rename(columns={'Geography':'Name','Type':'Type',2008:'Pop_08',2009:'Pop_09',2010:'Pop_10',
                            2011:'Pop_11',2012:'Pop_12',2013:'Pop_13',2014:'Pop_14',2015:'Pop_15',
                           2016:'Pop_16',2017:'Pop_17',2018:'Pop_18'})
total_df.head()

Year,Name,Pop_08,Pop_09,Pop_10,Pop_11,Pop_12,Pop_13,Pop_14,Pop_15,Pop_16,Pop_17,Pop_18,Type
0,United States,"251,009,998","253,253,138","256,390,428","258,666,188","261,032,045","263,356,005","265,732,678","268,005,522","270,301,427","272,460,904","274,605,948",National
1,Alabama,"3,870,996","3,898,688","3,980,625","3,997,493","4,019,335","4,041,871","4,059,211","4,070,748","4,082,972","4,094,777","4,109,359",State
2,Alaska,"559,346","565,800","577,749","584,567","592,771","599,385","599,474","600,941","604,157","603,161","602,095",State
3,Arizona,"5,235,123","5,316,387","5,229,908","5,304,658","5,388,365","5,469,060","5,567,140","5,663,944","5,770,861","5,874,596","5,996,296",State
4,Arkansas,"2,357,363","2,374,089","2,407,988","2,426,548","2,438,682","2,449,550","2,459,895","2,470,760","2,482,955","2,495,932","2,508,847",State


In [48]:
all_pops = total_df
all_pops['Pop_08'] = all_pops['Pop_08'].str.replace(',','').astype(float)
all_pops['Pop_09'] = all_pops['Pop_09'].str.replace(',','').astype(float)
all_pops['Pop_10'] = all_pops['Pop_10'].str.replace(',','').astype(float)
all_pops['Pop_11'] = all_pops['Pop_11'].str.replace(',','').astype(float)
all_pops['Pop_12'] = all_pops['Pop_12'].str.replace(',','').astype(float)
all_pops['Pop_13'] = all_pops['Pop_13'].str.replace(',','').astype(float)
all_pops['Pop_14'] = all_pops['Pop_14'].str.replace(',','').astype(float)
all_pops['Pop_15'] = all_pops['Pop_15'].str.replace(',','').astype(float)
all_pops['Pop_16'] = all_pops['Pop_16'].str.replace(',','').astype(float)
all_pops['Pop_18'] = all_pops['Pop_18'].str.replace(',','').astype(float)
all_pops.head()

Year,Name,Pop_08,Pop_09,Pop_10,Pop_11,Pop_12,Pop_13,Pop_14,Pop_15,Pop_16,Pop_17,Pop_18,Type
0,United States,251009998.0,253253138.0,256390428.0,258666188.0,261032045.0,263356005.0,265732678.0,268005522.0,270301427.0,"272,460,904",274605948.0,National
1,Alabama,3870996.0,3898688.0,3980625.0,3997493.0,4019335.0,4041871.0,4059211.0,4070748.0,4082972.0,"4,094,777",4109359.0,State
2,Alaska,559346.0,565800.0,577749.0,584567.0,592771.0,599385.0,599474.0,600941.0,604157.0,"603,161",602095.0,State
3,Arizona,5235123.0,5316387.0,5229908.0,5304658.0,5388365.0,5469060.0,5567140.0,5663944.0,5770861.0,"5,874,596",5996296.0,State
4,Arkansas,2357363.0,2374089.0,2407988.0,2426548.0,2438682.0,2449550.0,2459895.0,2470760.0,2482955.0,"2,495,932",2508847.0,State
5,California,29836645.0,30167113.0,30753805.0,31068145.0,31393072.0,31722579.0,32069627.0,32390164.0,32663036.0,"32,888,662",33089613.0,State
6,Colorado,4049693.0,4120964.0,4152382.0,4220896.0,4290960.0,4367984.0,4447355.0,4542949.0,4632063.0,"4,710,781",4791744.0,State
7,Connecticut,2932812.0,2951882.0,3013777.0,3029649.0,3043910.0,3054288.0,3061978.0,3061946.0,3060226.0,"3,060,949",3065086.0,State
8,Delaware,728265.0,736007.0,752949.0,760646.0,768406.0,777619.0,786714.0,795441.0,803341.0,"811,576",821504.0,State
9,District of Columbia,508021.0,515989.0,530795.0,541801.0,552642.0,564030.0,572788.0,582251.0,590875.0,"596,785",601238.0,State


In [49]:
all_pops_df = all_pops[['Name','Type','Pop_08','Pop_09','Pop_10','Pop_11','Pop_12','Pop_13','Pop_14','Pop_15',
                    'Pop_16','Pop_17','Pop_18',]]
all_pops_df.head()

Year,Name,Type,Pop_08,Pop_09,Pop_10,Pop_11,Pop_12,Pop_13,Pop_14,Pop_15,Pop_16,Pop_17,Pop_18
0,United States,National,251009998.0,253253138.0,256390428.0,258666188.0,261032045.0,263356005.0,265732678.0,268005522.0,270301427.0,"272,460,904",274605948.0
1,Alabama,State,3870996.0,3898688.0,3980625.0,3997493.0,4019335.0,4041871.0,4059211.0,4070748.0,4082972.0,"4,094,777",4109359.0
2,Alaska,State,559346.0,565800.0,577749.0,584567.0,592771.0,599385.0,599474.0,600941.0,604157.0,"603,161",602095.0
3,Arizona,State,5235123.0,5316387.0,5229908.0,5304658.0,5388365.0,5469060.0,5567140.0,5663944.0,5770861.0,"5,874,596",5996296.0
4,Arkansas,State,2357363.0,2374089.0,2407988.0,2426548.0,2438682.0,2449550.0,2459895.0,2470760.0,2482955.0,"2,495,932",2508847.0


In [50]:
all_pops_df.to_csv('overall_pop.csv', index=False)